In [1]:
import time
import glob
import os
import re
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

print(f'pandas: {pd.__version__}')
print(f'numpy: {np.__version__}')
print(f'matplotlib: {matplotlib.__version__}')
print(f'seaborn: {sns.__version__}')

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', str)

pandas: 2.1.1
numpy: 1.26.0
matplotlib: 3.8.0
seaborn: 0.13.0


# Load National Public Transport Access Nodes (NaPTAN) Data

The National Public Transport Access Nodes (NaPTAN) Data is available from the [NaPTAN website](https://beta-naptan.dft.gov.uk/download). It is a collection of public transport stops, stations and other access points in England, Scotland and Wales. It does not cover transport services in Northern Ireland. It is [updated daily](https://www.data.gov.uk/dataset/ff93ffc1-6656-47d8-9155-85ea0b8f2251/national-public-transport-access-nodes-naptan) and is available in a variety of formats. The full spec can be found under [NaPTAN guide for data managers](https://www.gov.uk/government/publications/national-public-transport-access-node-schema/naptan-guide-for-data-managers) The data is available under the [Open Government Licence 3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/).

Although the dataset contains both the British National Grid system (OSGB36) and the ETRS89 Geodetic Coordinate system information for each stop, we found that only easting and northing are determined for all stops. Many records are missing their ETRS89 Geodetic Coordinate system information. As we predominantly use the geographical coordinates for our analysis, we will convert the easting and northing information the latitude and longitude for each stop. To do that, we will use Ordnance Survey's desktop tool [Grid InQuestII](https://www.ordnancesurvey.co.uk/business-government/tools-support/os-net/transformation)

In [2]:
from modules.utils import load_data

def load_transport_data():
    saved_name = 'NaPTAN_stops_geodetic'

    columns = ['ATCOCode', 'CommonName', 'ShortCommonName', 'Landmark', 'Street', 'Indicator', 'Bearing', 'LocalityName', 'ParentLocalityName', 'Town', 'Suburb', 'LocalityCentre', 'Easting', 'Northing', 'StopType', 'BusStopType', 'CreationDateTime', 'ModificationDateTime', 'RevisionNumber', 'Modification', 'Status', 'ETRS89GD-Lat', 'ETRS89GD-Long']

    df = load_data(
        saved_name,
        lambda f: pd.read_csv(
            f, header=0, usecols=columns, low_memory=False
        )
    )

    if df is None:
        raise Exception('No data')

    # Convert the 'CreationDateTime' and 'ModificationDateTime' columns to datetime
    df['CreationDateTime'] = pd.to_datetime(df['CreationDateTime'], errors='coerce')
    df['ModificationDateTime'] = pd.to_datetime(df['ModificationDateTime'], errors='coerce')

    df.rename(columns={'ETRS89GD-Lat': 'Latitude', 'ETRS89GD-Long': 'Longitude'}, inplace=True)

    # Drop inactive stops
    df = df[df['Status'] == 'active']

    df = df.add_prefix('NPT_')
    df.set_index('NPT_ATCOCode', inplace=True)

    return df

df = load_transport_data()

Loading saved data from ./data/saved/NaPTAN_stops_geodetic.parquet...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386471 entries, 0100BRP90310 to 9400ZZTWWJU2
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NPT_CommonName            386471 non-null  object        
 1   NPT_ShortCommonName       89263 non-null   object        
 2   NPT_Landmark              254581 non-null  object        
 3   NPT_Street                373901 non-null  object        
 4   NPT_Indicator             367978 non-null  object        
 5   NPT_Bearing               366796 non-null  object        
 6   NPT_LocalityName          386470 non-null  object        
 7   NPT_ParentLocalityName    168861 non-null  object        
 8   NPT_Town                  137477 non-null  object        
 9   NPT_Suburb                63491 non-null   object        
 10  NPT_LocalityCentre        382018 non-null  object        
 11  NPT_Easting               386471 non-null  int64     

The initial dataset contains `386,471` records, however, we only need stops for London. As a rough estimate, we will find the latitude and longitude for the centre of London ([51.509865, -0.118092](https://www.latlong.net/place/london-uk-14153.html)) and use a [30km radius](https://www.researchgate.net/publication/321783807_Effect_of_night-time_temperatures_on_cause_and_age-specific_mortality_in_London#pf3) to filter the dataset. We also only consider active stops. The final dataset contains `28,016` records, a `92.75%` reduction in size.

In [4]:
from geopy.distance import geodesic

def vectorized_distances(central_point, latitudes, longitudes):
    return np.array([geodesic(central_point, (lat, lon)).meters for lat, lon in zip(latitudes, longitudes)])

def get_stops_within_radius(df, central_point, radius, types=None):
    if types:
        df = df[df['NPT_StopType'].isin(types)]
    
    distances = vectorized_distances(central_point, df['NPT_Latitude'], df['NPT_Longitude'])
    mask = distances <= radius
    return df[mask]

In [5]:
get_stops_within_radius(df=df, central_point=(51.39798, 0.00857), radius=200)[[
  'NPT_CommonName',
  'NPT_Street',
  'NPT_Bearing',
  'NPT_LocalityName',
  'NPT_ParentLocalityName',
  'NPT_StopType',
  'NPT_Latitude',
  'NPT_Longitude']]

,NPT_CommonName,NPT_Street,NPT_Bearing,NPT_LocalityName,NPT_ParentLocalityName,NPT_StopType,NPT_Latitude,NPT_Longitude
NPT_ATCOCode,,,,,,,,
490005900S,Highfield Drive,CUMBERLAND ROAD,S,Shortlands,Bromley,BCT,51.39840251,0.00624543
490005901N,Durham Avenue Shortlands,---,N,Shortlands,Bromley,BCT,51.3972761,0.0074325
490016066S,Durham Avenue Shortlands,CUMBERLAND ROAD,S,Shortlands,Bromley,BCT,51.39763997,0.00718965


The problem with this implementation is that it is very slow. It takes around 3s to search for all stops for one property to run the code. We will use the `BallTree` algorithm from the scikit-learn library to search for nearby points more efficiently. BallTree is designed for efficient distance-based queries and is more suitable for finding nearby records than calculating pairwise distances.

In [6]:
from sklearn.neighbors import BallTree

def get_stops_within_radius_improved(df, central_point, radius):
    rad_df = np.radians(df[['NPT_Latitude', 'NPT_Longitude']])

    tree = BallTree(rad_df, metric='haversine')

    # Define the search radius, with the Earth's radius in meters
    search_radius = radius / 6371e3

    # Calculate the count of nearby records
    nearby_points = tree.query_radius(np.radians(pd.DataFrame(central_point)).values.reshape(1, -1), r=search_radius)

    # Return the nearby records
    return pd.DataFrame([df.loc[rad_df.iloc[indices].name] for indices in nearby_points[0]])

In [7]:
get_stops_within_radius_improved(df=df, central_point=(51.39798, 0.00857), radius=200)[[
  'NPT_CommonName',
  'NPT_Street',
  'NPT_Bearing',
  'NPT_LocalityName',
  'NPT_ParentLocalityName',
  'NPT_StopType',
  'NPT_Latitude',
  'NPT_Longitude']]

,NPT_CommonName,NPT_Street,NPT_Bearing,NPT_LocalityName,NPT_ParentLocalityName,NPT_StopType,NPT_Latitude,NPT_Longitude
490016066S,Durham Avenue Shortlands,CUMBERLAND ROAD,S,Shortlands,Bromley,BCT,51.39763997,0.00718965
490005901N,Durham Avenue Shortlands,---,N,Shortlands,Bromley,BCT,51.3972761,0.0074325
490005900S,Highfield Drive,CUMBERLAND ROAD,S,Shortlands,Bromley,BCT,51.39840251,0.00624543


In [8]:
filtered_df = get_stops_within_radius_improved(df=df, central_point=(51.509865, -0.118092), radius=30000)

In [9]:
filtered_df['NPT_StopType'].value_counts()

NPT_StopType
BCT    24983
PLT      716
TMU      648
RSE      647
RLY      443
MET      357
FBT       85
FER       49
BCS       46
FTD       43
BCQ       15
TXR       10
GAT        8
Name: count, dtype: int64

# Enrich Merged Price Paid Data

We will use a similar technique to entrich the merged price paid data with the NaPTAN data. We will find all bus stops within the walking distance, all tube stops within 10 minutes walk or all tain stops within a 1.2km radius of each property and add the tally to the dataset. Note that this is a very rough estimate of the walking distance, it is using radius to find the nearest stops, the actual walking distance is likely to be longer.

As suggested by NIH, the walking distance is [`0.25 miles`](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3377942/#:~:text=Walking%20distance%20is%20an%20important,distance%20in%20U.S.%20research%20studies.) or `402 meters`. Although the actual speed of walking is variable, the Department of Transport suggests that a 10 minute walk is about [half a mile or `804 meters`](https://www.tpl.org/blog/why-the-10-minute-walk#:~:text=Though%20walking%20speeds%20vary%2C%20the,mile%20in%20about%2010%20minutes.).

A full list of [stop types](http://naptan.dft.gov.uk/naptan/schema/2.4/doc/NaPTANSchemaGuide-2.4-v0.57.pdf) (p94)

Even with the improved algorithm, this process is still quite intensive. It takes around `13m 25s` to search for all `4643` properties in Bromley alone, `35m 13.6s` to search for `12,659` properties, `53m 56.9s` for `20,649` and `3h 32m 19s` for `55,446` properties!! We will save the results to a csv file and load it back in the next notebook.

In [10]:
from modules.utils import load_saved_data
ppd_df = load_saved_data('1_ppd_epc_data')

Loading saved data from ./data/saved/1_ppd_epc_data.parquet...


In [11]:
busStopTypes =['BCT', 'BCS', 'BCQ']
railStopTypes = ['RSE', 'RLY']
tramMetroStopTypes = ['PLT', 'TMU', 'MET']
ferryStopTypes = ['FBT', 'FER', 'FTD']
taxiStopTypes = ['TXR']
airportStopTypes = ['GAT']

In [12]:
# test_df = ppd_df[ppd_df['PPD_ID'] == '{7C2D0700-9419-4963-E053-6B04A8C07B97}']
# test_df = ppd_df.loc['{666758D7-3B18-3363-E053-6B04A8C0D74E}']
# test_df.info()

In [13]:
import modules.utils as utils

def enrich_with_transport_data(ppd_df, npt_df):
    ppd_df = ppd_df[ppd_df['UPRN_LATITUDE'].notna()].reset_index()[['PPD_ID', 'UPRN_LATITUDE', 'UPRN_LONGITUDE']]

    type_map = {
        'NPT_NearbyBusStops': busStopTypes,
        'NPT_NearbyTramMetroStops': tramMetroStopTypes,
        'NPT_NearbyRailStops': railStopTypes,
    }
    
    stop_counts = []
    with utils.Timer() as t:
        for index, row in ppd_df.iterrows():
            nearby_stops = get_stops_within_radius_improved(df=npt_df, central_point=(row['UPRN_LATITUDE'], row['UPRN_LONGITUDE']), radius=804)

            if nearby_stops.empty:
                counts = {key: 0 for key in type_map.keys()}
            else:
                counts = {key: nearby_stops[nearby_stops['NPT_StopType'].isin(types)].shape[0] for key, types in type_map.items()}
            
            counts['NPT_NearbyStops'] = sum(counts.values())
            stop_counts.append(counts) 

            if (index % 1000) == 0:
                t.log(f'Processed {index}/{len(ppd_df)}')     

    stop_counts_df = pd.DataFrame(stop_counts)
    ppd_df = pd.concat([ppd_df, stop_counts_df], axis=1)

    ppd_df.drop(columns=['UPRN_LATITUDE', 'UPRN_LONGITUDE'], inplace=True) 
    ppd_df.set_index('PPD_ID', inplace=True)
    
    return ppd_df

# 123m 47.2s
enriched_ppd_df = enrich_with_transport_data(ppd_df, filtered_df)

=== Processed 0/388451 (0.0447s) ===
=== Processed 1000/388451 (16.9566s) ===
=== Processed 2000/388451 (33.3050s) ===
=== Processed 3000/388451 (49.2171s) ===
=== Processed 4000/388451 (65.0800s) ===
=== Processed 5000/388451 (80.9206s) ===
=== Processed 6000/388451 (96.3319s) ===
=== Processed 7000/388451 (111.8196s) ===
=== Processed 8000/388451 (127.4576s) ===
=== Processed 9000/388451 (143.5469s) ===
=== Processed 10000/388451 (159.5553s) ===
=== Processed 11000/388451 (175.8731s) ===
=== Processed 12000/388451 (191.7869s) ===
=== Processed 13000/388451 (207.5275s) ===
=== Processed 14000/388451 (223.2054s) ===
=== Processed 15000/388451 (238.9546s) ===
=== Processed 16000/388451 (254.8816s) ===
=== Processed 17000/388451 (270.9323s) ===
=== Processed 18000/388451 (286.6186s) ===
=== Processed 19000/388451 (302.1949s) ===
=== Processed 20000/388451 (318.2377s) ===
=== Processed 21000/388451 (333.9605s) ===
=== Processed 22000/388451 (349.3866s) ===
=== Processed 23000/388451 (364.

In [14]:
result_df = get_stops_within_radius_improved(df, central_point=(51.407734,	-0.032874), radius=200)
result_df.head()

,NPT_CommonName,NPT_ShortCommonName,NPT_Landmark,NPT_Street,NPT_Indicator,NPT_Bearing,NPT_LocalityName,NPT_ParentLocalityName,NPT_Town,NPT_Suburb,NPT_LocalityCentre,NPT_Easting,NPT_Northing,NPT_StopType,NPT_BusStopType,NPT_CreationDateTime,NPT_ModificationDateTime,NPT_RevisionNumber,NPT_Modification,NPT_Status,NPT_Latitude,NPT_Longitude
490015349E1,Hayne Road Beckenham,None,---,BECKENHAM ROAD,Stop BE,E,Beckenham,London,None,None,false,536737,169490,BCT,MKD,2016-08-05 14:47:06,2016-08-05 14:47:06,0.0,revise,active,51.40795633,-0.03538058
490014036W,Vicarage Drive,None,---,RECTORY ROAD,Stop W,S,Beckenham,London,None,None,false,537040,169618,BCT,MKD,2018-05-18 13:15:08,2018-05-18 13:15:08,0.0,revise,active,51.40903356,-0.03097743
490003743S,Croydon Road War Memorial,None,---,---,Stop S,S,Beckenham,London,None,None,false,536933,169342,BCT,MKD,2016-06-20 10:09:08,2016-06-20 10:09:08,0.0,revise,active,51.40657911,-0.03262136
490014036V,Vicarage Drive,None,---,RECTORY ROAD,Stop V,N,Beckenham,London,None,None,false,537030,169628,BCT,MKD,2018-05-18 13:15:08,2018-05-18 13:15:08,0.0,revise,active,51.40912584,-0.03111727
490003744R,Beckenham High Street War Memorial,None,---,---,Stop R,W,Beckenham,London,None,None,false,537025,169361,BCT,MKD,2018-10-25 08:48:02,2018-10-25 08:48:02,0.0,revise,active,51.40672767,-0.03129214


In [15]:
enriched_ppd_df[enriched_ppd_df['NPT_NearbyStops'].isna()]

,NPT_NearbyBusStops,NPT_NearbyTramMetroStops,NPT_NearbyRailStops,NPT_NearbyStops
PPD_ID,,,,


In [16]:
from modules.utils import save_data

# Save it for later
save_data(enriched_ppd_df, '3_ppd_transport_data')

Saving data to ./data/saved/3_ppd_transport_data.parquet...


In [17]:
len(enriched_ppd_df)

388451